In [1]:
import pandas as pd
import torchaudio
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from transformers import  AutoProcessor 


/home/prsood/projects/def-whkchun/prsood/sarcasm_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle("/project/def-whkchun/prsood/multi-modal-emotion/data/text_audio_video_emotion_data.pkl")

In [3]:
audio_path1 = "../data/train_splits_wav/dia71_utt0.wav"

# audio_path2 = "../data/val_splits_wav/dia113_utt9.wav"

def speech_file_to_array_fn(path , target_sampling_rate = 16000):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze()
    return torch.mean(speech, dim=0) if len(speech.shape) > 1 else speech # over the channel dimension

In [4]:
PROC = AutoProcessor.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")
speech_list = [speech_file_to_array_fn(audio_path1)]
numpy_speech_list = [item.numpy() for item in speech_list] 

speech_list_mask = torch.Tensor(np.array(PROC( numpy_speech_list, sampling_rate = 16000 , padding = True)['attention_mask']))
speech_list_input_values = pad_sequence(speech_list , batch_first = True, padding_value=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
speech_list_mask , speech_list_mask.shape

(tensor([[1., 1., 1.,  ..., 1., 1., 1.]]), torch.Size([1, 1024]))

In [15]:
speech_list[1].shape

torch.Size([17408])

In [11]:
(speech_list_mask[0] == 0).nonzero(as_tuple=True)[0] , (speech_list_mask[1] == 0).nonzero(as_tuple=True)[0]

(tensor([], dtype=torch.int64),
 tensor([17408, 17409, 17410,  ..., 24232, 24233, 24234]))

In [10]:
speech_list_input_values , speech_list_input_values.shape

(tensor([[ 0.0206,  0.0559,  0.0851,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0044, -0.0066, -0.0075,  ...,  0.0000,  0.0000,  0.0000]]),
 torch.Size([2, 24235]))

In [6]:
# df_train = df[df["split"] == "train"]
df.sort_values(by=['audio_shape'], ascending=True)

,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
6282,1,406,2,"hey , pheebs !",4,train,../../data/train_splits_wav/dia406_utt2.wav,dia406_utt2,joy,6,1024,6144,../../data/train_video/dia406_utt2.mp4,positive,2,None
7384,1,523,10,hi pheebs !,3,train,../../data/train_splits_wav/dia523_utt10.wav,dia523_utt10,joy,6,1024,6144,../../data/train_video/dia523_utt10.mp4,positive,2,None
13456,3,108,5,ow !,2,val,../../data/val_splits_wav/dia108_utt5.wav,dia108_utt5,anger,6,1024,6144,../../data/val_video/dia108_utt5.mp4,negative,0,"(0.0, 0.064)"
5666,0,332,1,shh,1,train,../../data/train_splits_wav/dia332_utt1.wav,dia332_utt1,neutral,2,1024,2048,../../data/train_video/dia332_utt1.mp4,neutral,1,None
4220,0,179,8,sagittarius ?,2,train,../../data/train_splits_wav/dia179_utt8.wav,dia179_utt8,neutral,6,1024,6144,../../data/train_video/dia179_utt8.mp4,neutral,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9542,1,748,0,oh yes ! thank you very much ! oh that s that ...,19,train,../../data/train_splits_wav/dia748_utt0.wav,dia748_utt0,joy,2,257024,514048,../../data/train_video/dia748_utt0.mp4,positive,2,"(0.040059850374064836, 2.043052369077307)"
8626,2,649,0,joey ? could you get that ? what are you doing...,21,train,../../data/train_splits_wav/dia649_utt0.wav,dia649_utt0,surprise,2,281259,562518,../../data/train_video/dia649_utt0.mp4,positive,2,"(0.1001064208428246, 17.33843208997722)"
7341,1,517,6,"yeah , say my name say it ! and now i m going ...",27,train,../../data/train_splits_wav/dia517_utt6.wav,dia517_utt6,joy,2,322219,644438,../../data/train_video/dia517_utt6.mp4,positive,2,"(0.7206687375745526, 16.3551766277336)"
13141,0,74,7,"okay uh , ross and rachel rachel and ross that...",22,val,../../data/val_splits_wav/dia74_utt7.wav,dia74_utt7,neutral,2,335872,671744,../../data/val_video/dia74_utt7.mp4,neutral,1,"(0.9605490943755958, 19.51115347950429)"


,emotion,dialog,utterance,text,num_words,split,audio_path,name,emotion_label,num_channels,audio_shape,size_padding,video_path,sentiment_label,sentiment,timings
1578,0,169,16,yeah,1,test,../../data/test_splits_wav/dia169_utt16.wav,dia169_utt16,neutral,2,9899,19798,../../data/test_video/dia169_utt16.mp4,neutral,1,"(0.30934375, 0.4124583333333333)"
2153,3,234,5,"you were gonna spill hot coffee all over me , ...",14,test,../../data/test_splits_wav/dia234_utt5.wav,dia234_utt5,anger,6,9899,59394,../../data/test_video/dia234_utt5.mp4,negative,0,None
4219,0,179,7,nope,1,train,../../data/train_splits_wav/dia179_utt7.wav,dia179_utt7,neutral,6,9899,59394,../../data/train_video/dia179_utt7.mp4,neutral,1,"(0.3918354166666666, 0.4743270833333333)"
4216,0,179,4,taurus ?,2,train,../../data/train_splits_wav/dia179_utt4.wav,dia179_utt4,neutral,6,9899,59394,../../data/train_video/dia179_utt4.mp4,neutral,1,"(0.18560625, 0.30934375)"
10400,4,841,6,right here !,3,train,../../data/train_splits_wav/dia841_utt6.wav,dia841_utt6,sadness,6,9899,59394,../../data/train_video/dia841_utt6.mp4,negative,0,"(0.08249166666666666, 0.5568187499999999)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13398,2,99,3,ooohhhh ! !,3,val,../../data/val_splits_wav/dia99_utt3.wav,dia99_utt3,surprise,6,1024,6144,../../data/val_video/dia99_utt3.mp4,positive,2,None
5666,0,332,1,shh,1,train,../../data/train_splits_wav/dia332_utt1.wav,dia332_utt1,neutral,2,1024,2048,../../data/train_video/dia332_utt1.mp4,neutral,1,None
3293,0,71,0,pheebs ?,2,train,../../data/train_splits_wav/dia71_utt0.wav,dia71_utt0,neutral,6,1024,6144,../../data/train_video/dia71_utt0.mp4,neutral,1,None
7384,1,523,10,hi pheebs !,3,train,../../data/train_splits_wav/dia523_utt10.wav,dia523_utt10,joy,6,1024,6144,../../data/train_video/dia523_utt10.mp4,positive,2,None
